<a href="https://colab.research.google.com/github/todebode/Food-sales-predictions/blob/main/Food_sales_predictions_Part_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 1 - Part 5 (Core)

Tijesunimi Odebode

02/27/2022

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### 1. Import Necessary Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn import set_config
set_config(display='diagram')

### 2. Load the Data

In [ ]:
filename = "/content/drive/MyDrive/Coding Dojo Bootcamp/Data/sales_predictions.csv"
sales_predictions = pd.read_csv(filename)
sales_predictions.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


### 3. Explore the data

In [ ]:
sales_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8523 entries, 0 to 8522
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Item_Identifier            8523 non-null   object 
 1   Item_Weight                7060 non-null   float64
 2   Item_Fat_Content           8523 non-null   object 
 3   Item_Visibility            8523 non-null   float64
 4   Item_Type                  8523 non-null   object 
 5   Item_MRP                   8523 non-null   float64
 6   Outlet_Identifier          8523 non-null   object 
 7   Outlet_Establishment_Year  8523 non-null   int64  
 8   Outlet_Size                6113 non-null   object 
 9   Outlet_Location_Type       8523 non-null   object 
 10  Outlet_Type                8523 non-null   object 
 11  Item_Outlet_Sales          8523 non-null   float64
dtypes: float64(4), int64(1), object(7)
memory usage: 799.2+ KB


- As seen above, there are two columns with missing data. The columns are '**Item_Weight**' and '**Outlet_Size**.'

### 3. Identify each feature as numerical, ordinal, or nominal.

**Numerical features:** 'Item_Weight', 'Item_Visibility', 'Item_MRP'

**Nominal features:** 'Item_Identifier', 'Item_Type', 'Outlet_Identifier', 'Outlet_Type'

**Ordinal features:** 'Item_Fat_Content', 'Outlet_Establishment_Year', 'Outlet_Size', 'Outlet_Location_Type'

### 4. Ordinal Encoding

We will ordinal encode all ordinal features

In [ ]:
# Let's see the unique values in the 'Item_Fat_Content' column 
sales_predictions['Item_Fat_Content'].value_counts()

Low Fat    5089
Regular    2889
LF          316
reg         117
low fat     112
Name: Item_Fat_Content, dtype: int64

- We see that there are some inconsistent values. "Low Fat" is written in different ways - capitalized, lower case and even abbreviated. 'Regular' is written in two ways: 'Regular' and 'reg.' We will correct those in the same step that we ordinal encode this column

In [ ]:
# Ordinal Encode 'Item_Fat_Content'
replacement_dictionary = {'Regular':1, 'reg':1, 'Low Fat':0, 'LF':0, 
                          'low fat':0}
sales_predictions['Item_Fat_Content'].replace(replacement_dictionary, 
                                              inplace=True)
sales_predictions['Item_Fat_Content'].value_counts()

0    5517
1    3006
Name: Item_Fat_Content, dtype: int64

In [ ]:
# Let's see the unique values in the 'Outlet_Establishment_Year' column 
sales_predictions['Outlet_Establishment_Year'].value_counts()

1985    1463
1987     932
1999     930
1997     930
2004     930
2002     929
2009     928
2007     926
1998     555
Name: Outlet_Establishment_Year, dtype: int64

In [ ]:
# Ordinal Encode 'Outlet_Establishment_Year'
replacement_dictionary = {2009:8, 2007:7, 2004:6, 2002:5, 1999:4, 1998:3, 
                          1997:2, 1987:1, 1985:0}
sales_predictions['Outlet_Establishment_Year'].replace(replacement_dictionary,
                                                       inplace=True)
sales_predictions['Outlet_Establishment_Year'].value_counts()

0    1463
1     932
4     930
2     930
6     930
5     929
8     928
7     926
3     555
Name: Outlet_Establishment_Year, dtype: int64

In [ ]:
# Let's see the unique values in the 'Outlet_Size' column 
sales_predictions['Outlet_Size'].value_counts()

Medium    2793
Small     2388
High       932
Name: Outlet_Size, dtype: int64

In [ ]:
# Ordinal Encode 'Outlet_Size'
replacement_dictionary = {'High':2, 'Medium':1, 'Small':0}
sales_predictions['Outlet_Size'].replace(replacement_dictionary, inplace=True)
sales_predictions['Outlet_Size'].value_counts()

1.0    2793
0.0    2388
2.0     932
Name: Outlet_Size, dtype: int64

In [ ]:
# Let's see the unique values in the 'Outlet_Location_Type' column 
sales_predictions['Outlet_Location_Type'].value_counts()

Tier 3    3350
Tier 2    2785
Tier 1    2388
Name: Outlet_Location_Type, dtype: int64

In [ ]:
# Ordinal Encode 'Outlet_Location_Type'
replacement_dictionary = {'Tier 3':2, 'Tier 2':1, 'Tier 1':0}
sales_predictions['Outlet_Location_Type'].replace(replacement_dictionary, inplace=True)
sales_predictions['Outlet_Location_Type'].value_counts()

2    3350
1    2785
0    2388
Name: Outlet_Location_Type, dtype: int64

### 5. Validation Split

In [ ]:
X = sales_predictions.drop(columns=['Item_Outlet_Sales'])
y = sales_predictions['Item_Outlet_Sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

### 6. Instantiate Column Selectors

In [ ]:
cat_selector = make_column_selector(dtype_include='object')
num_selector = make_column_selector(dtype_include='number')

### 7. Instantiate Transformers.

We will be using 3 different transformers: SimpleImputer, StandardScaler, and OneHotEncoder. There will be one SimpleImputer with one imputation strategy: ‘median’ strategy for numeric columns. The only categorical column with missing value has been ordinal encoded, so we won't need to use SimpleImputer for that column.

In [ ]:
# Imputers
median_imputer = SimpleImputer(strategy='median')
# Scaler
scaler = StandardScaler()
# One-hot encoder
ohe = OneHotEncoder(handle_unknown='ignore', sparse=False)

### 8. Instantiate Pipelines

We will be using TWO different pipelines. One for numeric data and one for nominal categorical data.

In [ ]:
# Numeric pipeline
numeric_pipe = make_pipeline(median_imputer, scaler)
numeric_pipe

Pipeline(steps=[('simpleimputer', SimpleImputer(strategy='median')),
                ('standardscaler', StandardScaler())])

In [ ]:
# # Nominal Categorical pipeline
# categorical_pipe = make_pipeline(ohe)
# categorical_pipe

### 9. Instantiate ColumnTransformer

In [ ]:
# Tuples for Column Transformer
number_tuple = (numeric_pipe, num_selector)
category_tuple = (ohe, cat_selector)
# ColumnTransformer
preprocessor = make_column_transformer(number_tuple, category_tuple)
preprocessor

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb55912a850>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb55912a890>)])

### 10. Fit the ColumnTransformer on the training data and transform both training and testing data

In [ ]:
preprocessor.fit(X_train)

ColumnTransformer(transformers=[('pipeline',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('standardscaler',
                                                  StandardScaler())]),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb55912a850>),
                                ('onehotencoder',
                                 OneHotEncoder(handle_unknown='ignore',
                                               sparse=False),
                                 <sklearn.compose._column_transformer.make_column_selector object at 0x7fb55912a890>)])

In [ ]:
# transform training and testing data
X_train_processed = preprocessor.transform(X_train)
X_test_processed = preprocessor.transform(X_test)

### 11. Inspect the Result

In [ ]:
print(np.isnan(X_train_processed).sum().sum(), 'missing values in training data')
print(np.isnan(X_test_processed).sum().sum(), 'missing values in testing data')
print('\n')
print('All data in X_train_processed are', X_train_processed.dtype)
print('All data in X_test_processed are', X_test_processed.dtype)
print('\n')
print('shape of data is', X_train_processed.shape)
print('\n')
X_train_processed

0 missing values in training data
0 missing values in testing data


All data in X_train_processed are float64
All data in X_test_processed are float64


shape of data is (6392, 1587)




array([[ 0.82748547, -0.7403206 , -0.71277507, ...,  0.        ,
         1.        ,  0.        ],
       [ 0.56664432,  1.35076614, -1.29105225, ...,  0.        ,
         1.        ,  0.        ],
       [-0.12102782,  1.35076614,  1.81331864, ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 1.12389588, -0.7403206 , -0.92052713, ...,  1.        ,
         0.        ,  0.        ],
       [ 1.77599877, -0.7403206 , -0.2277552 , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.82748547, -0.7403206 , -0.95867683, ...,  1.        ,
         0.        ,  0.        ]])

- From the above, we see that there are no missing data, all data are of type float64, and there are 1587 columns now.